## Data Cleaning Plan

We have three data sets:
- Markers' bios and metadata (markers_bios)
- Followers' bios and metadata (followers_bios)
- All brands and their followers (markers-followers)


Step by step plan:
1. Load the bios of followers, and the marker-follower file. 
    - Provide summary statistics of users and brands. How many brands do we have? How many followers? Any missing data, duplicates etc.?

2. Filter on marker-follower df:
    - Create a dictionary of counts brands per follower
    - Remove users that follow less than 5 (or more) brands
    - Continuously track numbers of users removed
    - Match the Follower_Ids in the now filtered marker-follower df with the follower-bio df. As such, the follower bios will only include users that follow more than five brands. Subsequent filters will be on the correct users (up to date follower-bios).

3. Do the filters on the follower-bios:
    - Remove users with less than 25 followers
    - Remove users with less than 100 tweets

4. Filter based on language: keep only french accounts









In [1]:
# Standard library imports
from collections import defaultdict
from datetime import datetime
import csv
import html
import os
import re
import sys

# Third-party library imports
import dask.dataframe as dd
from joblib import Parallel, delayed
from langdetect import detect_langs, LangDetectException, DetectorFactory
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import psutil
import pickle
import regex
import seaborn as sns
from unidecode import unidecode
import gcld3
import ftfy
import importlib

# Local application/library specific imports
import utils2
from utils2 import *

## 1. Load files and summary stats

In [7]:
# # Load the data files and rename ID columns
importlib.reload(utils2)

# Load markers-followers
load_path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_2023-05-19.csv'

req_cols = ['id', 'follower_id']
dtypes = {'id': 'object',
          'follower_id': 'object'}

markers_followers = utils2.fileloader(load_path, file, req_cols, dtypes)


#rename the twittwer id column to follower id 
markers_followers.rename(columns={'id':'marker_id'}, inplace=True)

In [8]:
# Load the followers bios and rename ID columns
load_path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_bios_2023-05-19.csv'

req_cols = ['twitter_id', 'id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

dtypes = {
    'twitter_id': 'object',
    'id': 'object',
    'screen_name': 'object',
    'description': 'object',
    'location': 'object',
    'tweets': 'float64',
    'followers': 'float64',
    'friends': 'float64',
    'witheld_in_countries': 'float64'
}

followers_bios = utils2.fileloader(load_path, file, req_cols, dtypes)



#rename the twittwer id column to follower id 
followers_bios.rename(columns={'twitter_id':'follower_id'}, inplace=True)

Summary statistics

In [9]:
utils2.summary_stats(followers_bios, print_dtypes=False)

Shape of DataFrame:  (70666646, 11)

Columns in DataFrame:  ['follower_id', 'id', 'screen_name', 'description', 'timestamp_utc', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists']

Number of unique values in 'follower_id':  70666646
Number of duplicate values in 'follower_id':  0

Number of unique values in 'id':  70642661
Number of duplicate values in 'id':  23984

Number of missing values in each column:
'follower_id':  0
'id':  23985
'screen_name':  23986
'description':  42027215
'timestamp_utc':  23985
'location':  47956041
'tweets':  23985
'followers':  23985
'friends':  23985
'likes':  23985
'lists':  23985

Number of duplicate rows:  0


In [10]:
importlib.reload(utils2)
utils2.summary_stats(markers_followers, print_dtypes=False)

Shape of DataFrame:  (126345412, 2)

Columns in DataFrame:  ['marker_id', 'follower_id']

Number of unique values in 'follower_id':  70636295
Number of duplicate values in 'follower_id':  55709117

Number of unique values in 'marker_id':  236
Number of duplicate values in 'marker_id':  126345176

Number of missing values in each column:
'marker_id':  0
'follower_id':  0

Number of duplicate rows:  2357493


In [11]:
# Sort the DataFrame to ensure that duplicates are next to each other
markers_followers_sorted = markers_followers.sort_values(by=list(markers_followers.columns))

# Find duplicates in the sorted DataFrame
duplicates = markers_followers_sorted[markers_followers_sorted.duplicated(keep=False)]

# Print the first 10 rows of duplicates (5 pairs)
print(duplicates.head(10))

         marker_id          follower_id
81032594  25053299            100000025
89256298  25053299            100000025
79323950  25053299  1000001004220420096
87547795  25053299  1000001004220420096
79369517  25053299  1000001771266232320
87593356  25053299  1000001771266232320
79562687  25053299           1000001815
87786516  25053299           1000001815
79371019  25053299  1000002790238777352
87594858  25053299  1000002790238777352


In [12]:
duplicates.shape

(4714986, 2)

In [6]:
#drop the duplicates in markers_followers
markers_followers.drop_duplicates(keep='first', inplace=True)

In [7]:
markers_followers.shape

(123987919, 2)

In [8]:
compare_column_values(followers_bios, markers_followers, 'follower_id')

There are 30351 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


## 2. Filter the marker-follower df

- Filter the marker-follower df:
    - Remove users that follow less than 5 (or more) brands

    - Continuously track numbers of users removed
    
    - Match the Follower_Ids in the now filtered marker-follower df with the follower-bio df. As such, the follower bios 
    will only include users that follow more than five brands. Subsequent filters will be on the correct users (up to date follower-bios).

Remove users that follow less than 5 brands

In [9]:
n = 5  # minimal number of brands followed required to be included in the analysis
importlib.reload(utils2)
markers_followers_5 = utils2.filter_followers(markers_followers, 'follower_id', n)

66693204 followers follow less than 5 brands (94.42% of the total followers).
After removing these followers, 3943091 followers are left (5.58% of the followers in the inputted df).


In [12]:
#number of unique brands left

# Get the unique marker_id values in the original and filtered DataFrames
original_brands = set(markers_followers['marker_id'].unique())
filtered_brands = set(markers_followers_5['marker_id'].unique())

# Find the brands that are in the original DataFrame but not in the filtered DataFrame
removed_brands = original_brands - filtered_brands

# Print the removed brands
print("Removed brands:", removed_brands) #corresponds to "Napapijiri97", which kindof sounds like a fake profile


Removed brands: {'1059975643'}


Match the IDs in the filtered marker-follower df with the follower bio df, so that the follower bios only are for those who follow at least 5 brands

In [13]:
followers_bios_5 = utils2.streamline_IDs(source = markers_followers_5, df_tofilter= followers_bios, 'follower_id')

Number of unique follower_id in source DataFrame: 3943091
Number of unique follower_id in filtered DataFrame after filtering: 3943091
Removed 66723555 rows from the DataFrame to be filtered.
3943091 rows are left in the filtered DataFrame.


In [14]:
compare_column_values(followers_bios_5, markers_followers_5, 'follower_id')   

There are 0 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


## 3. Do the filters on the follower-bios:
- Remove users with less than 25 followers
- Remove users with less than 100 tweets
- Update the markers-followers df to match the now filtered bio df
- Filter based on language: keep only french accounts


In [15]:

followers_bios_fullfilter = utils2.filter_by_tweets_and_followers(followers_bios_5, min_followers= 25, min_tweets= 100)


Removed 2750559 rows.
1192532 rows are left.


Again, remove the follower_Ids in markers-followers that don't occur in the newly filtered  followers_bios_nd5_tweets_followers

In [16]:
markers_followers_fullfilter = utils2.streamline_IDs(source= followers_bios_fullfilter, df_tofilter=markers_followers_5, column='follower_id')

Number of unique follower_id in source DataFrame: 1192532
Number of unique follower_id in filtered DataFrame after filtering: 1192532
Removed 18992709 rows from the DataFrame to be filtered.
9614122 rows are left in the filtered DataFrame.


In [17]:
compare_column_values(followers_bios_fullfilter, markers_followers_fullfilter , 'follower_id')

There are 0 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


In [19]:
#Before writing ti csv, clean description column to avoid writing problems
importlib.reload(utils2)

followers_bios_fullfilter = utils2.process_description(followers_bios_fullfilter, 'description')

In [21]:
summary_stats(markers_followers_fullfilter, print_dtypes=False)

Shape of DataFrame:  (9614122, 2)

Columns in DataFrame:  ['marker_id', 'follower_id']



Number of unique values in 'follower_id':  1192532
Number of duplicate values in 'follower_id':  8421590

Number of unique values in 'marker_id':  235
Number of duplicate values in 'marker_id':  9613887

Number of missing values in each column:
'marker_id':  0
'follower_id':  0

Number of duplicate rows:  0


In [22]:
# # #Now write the two dfs to csvs to save them in case something happens
markers_followers_fullfilter.to_csv('/home/livtollanes/NewData/markers_followers_cleaned_nolang.csv', encoding='utf-8', index=False)

followers_bios_fullfilter.to_csv('/home/livtollanes/NewData/followers_bios_cleaned_nolang3.csv', sep=',', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)



## 4. Filter based on language: keep only french accounts
- Use language recognition alorithms to filter the follower_bios. 
- We only want french language bios to be included


In [15]:
import pandas as pd
import importlib

from langdetect import detect_langs, LangDetectException, DetectorFactory
import joblib
from joblib import Parallel, delayed

from langutils import *

In [7]:
#Load marker followers
full_path1 = '/home/livtollanes/NewData/markers_followers_cleaned_nolang.csv'
req_cols = ['marker_id', 'follower_id']
dtypes = {'marker_id': 'object',
          'follower_id': 'object'}

markers_followers_clean = pd.read_csv(full_path1, encoding='utf-8', dtype=dtypes, usecols=req_cols)

In [8]:
# #Loading the followers bios (with cleaned description column)
full_path = '/home/livtollanes/NewData/followers_bios_cleaned_nolang3.csv'

req_cols = ['follower_id', 'screen_name', 'description', 'description_cleantext', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

dtypes = {
    'follower_id': 'object',
    'screen_name': 'object',
    'description': 'object',
    'description_cleantext': 'object',
    'location': 'object',
    'tweets': 'float64',
    'followers': 'float64',
    'friends': 'float64'
}

follower_bios_cleaned3 = pd.read_csv(full_path, usecols=req_cols, dtype=dtypes, engine= 'python')

In [4]:
compare_column_values(follower_bios_cleaned3, markers_followers_clean, 'follower_id')

#The follower_ids are still streamlined, indicating that writing and reading of the cleaned dfs was successful

There are 0 unique values in df1 that don't exist in df2.
There are 0 unique values in df2 that don't exist in df1.


In [13]:
#should probably do summary stats on the dfs again, to make sure that no strange things have happened during the cleaning process
summary_stats(follower_bios_cleaned3, print_dtypes= False)

Shape of DataFrame:  (1192532, 11)

Columns in DataFrame:  ['follower_id', 'screen_name', 'description', 'timestamp_utc', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists', 'description_cleantext']

Number of unique values in 'follower_id':  1192532
Number of duplicate values in 'follower_id':  0

Number of missing values in each column:
'follower_id':  0
'screen_name':  0
'description':  296871
'timestamp_utc':  0
'location':  376685
'tweets':  0
'followers':  0
'friends':  0
'likes':  0
'lists':  0
'description_cleantext':  313934

Number of duplicate rows:  0


In [14]:
summary_stats(markers_followers_clean, print_dtypes= False)

Shape of DataFrame:  (9614122, 2)

Columns in DataFrame:  ['marker_id', 'follower_id']

Number of unique values in 'follower_id':  1192532
Number of duplicate values in 'follower_id':  8421590

Number of unique values in 'marker_id':  235
Number of duplicate values in 'marker_id':  9613887

Number of missing values in each column:
'marker_id':  0
'follower_id':  0

Number of duplicate rows:  0


Here comes the language detection

- First language detection is with the langdetect package. This package is based on Google's language detection API. On medium, it is actually reported that this package may not perform very accurate on short or mixed lnaguage texts, which is exactly what we are deailing with in twitter bios. 
- Gcld3 mmight be better at handling short text inputs like bios. By reading the documentation, gcld3 seems to be more fitting for social media data language detection

In [16]:
importlib.reload(langutils)
# Create a copy of the DataFrame for each function
follower_bios_cleaned3_copy1 = follower_bios_cleaned3.copy()
#follower_bios_cleaned3_copy2 = follower_bios_cleaned3.copy()

# Use the copied DataFrames in the functions
lang = langutils.add_and_detect_language(follower_bios_cleaned3_copy1, 'description_cleantext', seed = 3)
#lang2 = utils2.detect_language_gcld3(follower_bios_cleaned3_copy2, 'description_cleantext', n_jobs=-1)

In [51]:
#inspect the unique languages in lang
lang['language'].unique()

array(['en', 'es', 'fr', 'unknown', 'de', 'tr', 'ca', 'ja', 'cy', 'it',
       'no', 'pt', 'af', 'ar', 'el', 'et', 'so', 'ko', 'nl', 'pl', 'da',
       'tl', 'sw', 'ru', 'sv', 'sk', 'ro', 'id', 'sl', 'hr', 'fi', 'hu',
       'th', 'vi', 'mk', 'zh-cn', 'fa', 'hi', 'lt', 'ur', 'bn', 'lv',
       'sq', 'cs', 'bg', 'ne', 'he', 'ta', 'uk', 'mr', 'gu', 'pa',
       'zh-tw', 'ml', 'kn', 'te'], dtype=object)

In [56]:
total_rows = lang.shape[0]

french_rows = lang[lang['language'] == 'fr'].shape[0]
english_rows = lang[lang['language'] == 'en'].shape[0]
unknown_rows = lang[lang['language'] == 'unknown'].shape[0]
NA_rows = lang[lang['language'] == 'NA'].shape[0]
other_rows = total_rows - french_rows - english_rows - unknown_rows

french_percent = (french_rows / total_rows) * 100
english_percent = (english_rows / total_rows) * 100
unknown_percent = (unknown_rows / total_rows) * 100
other_percent = 100 - french_percent - english_percent - unknown_percent

print("French: ", french_rows, "(", french_percent, "%)")
print("English: ", english_rows, "(", english_percent, "%)")
print("Unknown: ", unknown_rows, "(", unknown_percent, "%)")
print("Other: ", other_rows, "(", other_percent, "%)")

nan_rows = lang['description_cleantext'].isna().sum()
nan_percent = (nan_rows / total_rows) * 100
print("NaN in description_cleantext: ", nan_rows, "(", nan_percent, "%)")

French:  268891 ( 22.54790647127289 %)
English:  353321 ( 29.627800344141708 %)
Unknown:  327271 ( 27.443372588743948 %)
Other:  243049 ( 20.380920595841467 %)
NaN in description_cleantext:  313934 ( 26.324995891095586 %)


In [17]:
total_rows = lang2.shape[0]

french_percent = (lang2[lang2['language'] == 'fr'].shape[0] / total_rows) * 100
english_percent = (lang2[lang2['language'] == 'en'].shape[0] / total_rows) * 100
unknown_percent = (lang2[lang2['language'] == 'unknown'].shape[0] / total_rows) * 100
other_percent = 100 - french_percent - english_percent - unknown_percent

print("French: ", french_percent, "%")
print("English: ", english_percent, "%")
print("Unknown: ", unknown_percent, "%")
print("Other: ", other_percent, "%")

nan_percent = (lang2['description_cleantext'].isna().sum() / total_rows) * 100
print("NaN in description_cleantext: ", nan_percent, "%")

French:  20.72791338094072 %
English:  21.173184451234853 %
Unknown:  26.324995891095586 %
Other:  31.77390627672884 %
NaN in description_cleantext:  26.324995891095586 %


### Manually creating test sets to compare the two language detection methods

For the langdetect df

In [ ]:
test1 = lang.sample(n=100, random_state=1)
test2 = lang2.sample(n=100, random_state=1)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
test1[['screen_name', 'location','description_cleantext', 'language']]

In [ ]:


true_labels = {
    'MorganJewelers1': 'non_fr',
    'LeighAnnTowne': 'non_fr',
    'ALBrutel': 'NA',
    'Fonkwadiour1': 'non_fr',
    'JUSTINAGROSSO': 'NA',
    'SarahRadif': 'non_fr',
    'CarineMayol': 'NA',
    'ManonMondou': 'fr',
    'erickleangar': 'NA',
    'elaineguimarae': 'non_fr',
    'danyunes': 'fr',
    'kuronapilled': 'non_fr',
    'KiriYaji': 'non_fr',
    'floriannisolle': 'fr',
    'Naman93726237': 'fr', #french but from guinea
    'doktor_vananga': 'non_fr', 
    'Slaiidh': "non_fr", #wrote non. could be another language
    'StarRaull': "non_fr",
    'SPeperstraete': 'non_fr',
    'yorgaine_lyon69': 'fr',
    'blinameta': 'non_fr',
    'AnnaRus75': 'non_fr',
    'BernTurner': 'NA',
    'lauriebeebe2': 'non_fr',
    'JeremyyPalazzo': 'NA',
    'GalerieCoulange': 'fr',
    'HaziqkaJ': 'NA',
    'DaraCheick': 'non_fr', #but seems to belong to a french uni
    'mmdlbdrny2': 'NA',
    'blandineleonie': 'fr',
    'CarlosO324': 'non_fr',
    'joshuatoney3': 'non_fr',
    'StephaneWayler': 'fr',
    'JessieV_214': 'non_fr',
    'Apartofsuzanne': 'non_fr',#but writes that she is french, and location is Paris
    'katuxka1': 'NA',
    'tamaitanya': 'NA',
    'sulagnasays': 'non_fr',
    'nicolesimon': 'non_fr',
    'lunasitah': 'non_fr',
    'ln971': 'fr',
    'olive_cas': 'non_fr',
    'Mercedeszatfc': 'NA',
    'hady_saad': 'non_fr',
    'GatienLeroux': 'fr',
    'Meluxiam': 'non_fr',
    'yulimadera10': 'non_fr',
    'RyanBrossault': 'fr',
    'FlotillaMRY': 'non_fr',
    'love98_daniela': 'non_fr',
    'jucag4115': 'NA',
    'Bxwiz': 'non_fr',
    'MLECOMTE': 'fr',
    'DelgadoJanahi': 'non_fr',
    'f1rmin': 'fr',
    'SylvainWalczak': 'NA',
    'SoyLigiawn': 'non_fr',
    'javielo26': 'non_fr',
    'CherrybangMUA': 'non_fr',
    '1927_albert': 'non_fr',
    'aayyoud': 'NA',
    'princessivy28': 'non_fr',
    'drumz420': 'NA',
    'Cjruin': 'non_fr',
    'xXallymayXx': 'non_fr',
    'NaAutacaace': 'fr',
    'RosaLuc15360022': 'non_fr',
    'PatrikWinston': 'non_fr',
    'mwa_sisqo': 'NA',
    'vankrug': 'NA',
    'miktoi': 'non_fr',
    'cescoeco': 'non_fr',
    'floo_ncy': 'NA',
    'Quentin_1411': 'non_fr',
    'Tvy_Tk': 'NA',
    'AlbanLeneveu': 'NA',
    'adh2311': 'non_fr',
    'loaizayose': 'NA',
    'mathilde_Fparis': 'fr',
    'gouillardmichae': 'fr',
    'RomainSprynski': 'fr',
    'styledscience': 'non_fr',
    'amhammadi': 'NA',
    'AliRazaSharif2': 'NA',
    'JahanLutz': 'fr',
    'HERVEJEHL': 'fr',
    'CatherineMSch': 'fr',
    'PoliKeyCo': 'non_fr',
    'HisGraceth': 'non_fr',
    'verstagoat': 'non_fr',
    'G_deLinares': 'fr', 
    'Granadechkou': 'fr',
    'MommaDandine': 'NA',
    'allionesolution': 'non_fr',
    'Ross75016': 'fr',
    'DorignyTheo': 'NA',
    'nikkolasg1': 'non_fr',
    'InesNau': 'fr',
    'FrenchEmperior': 'non_fr',
    'BekaEssick': 'non_fr'}

In [ ]:
def label_language(lang):
    if lang == 'unknown':
        return 'NA'
    elif lang == 'fr':
        return 'fr'
    else:
        return 'non_fr'

test1['pred_lang'] = test1['language'].apply(label_language)
test2['pred_lang'] = test2['language'].apply(label_language)

test1['true_lang'] = test1['screen_name'].map(true_labels)
test2['true_lang'] = test2['screen_name'].map(true_labels)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate metrics for test1
print("Metrics for test1:")
print(classification_report(test1['true_lang'], test1['pred_lang']))
print("Accuracy:", accuracy_score(test1['true_lang'], test1['pred_lang']))

# Calculate metrics for test2
print("\nMetrics for test2:")
print(classification_report(test2['true_lang'], test2['pred_lang']))
print("Accuracy:", accuracy_score(test2['true_lang'], test2['pred_lang']))

- Still lack the actual language filtering
- Think of a more sophisticated way to locate french users than only using language. This method should capture french users that write in english, and in french, and avoid selecting users that are writing in french but are not actually french. 

### FIltering french users based on location

In [45]:
# Use the function
utils2.location_bio_stats(follower_bios_cleaned3)

Unique locations: 231725 (19.4%)
Users with location data: 815847 (68.4%)
Users without location data: 376685 (31.6%)
Users with bios: 878598 (73.7%)
Users without bios: 313934 (26.3%)
Users with both location and bios: 668947 (56.1%)


There is a library called locationtagger. Might be useful

In [61]:
#check the type of the location column 
print(follower_bios_cleaned3['location'].dtypes)

object


In [60]:
follower_bios_cleaned3.columns

Index(['follower_id', 'screen_name', 'description', 'timestamp_utc',
       'location', 'tweets', 'followers', 'friends', 'likes', 'lists',
       'description_cleantext'],
      dtype='object')

In [34]:
#switch to geotext environment for this to work
# import geotext
# from geotext import GeoText
import nltk
import spacy
import locationtagger

In [32]:
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/livtollanes/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/livtollanes/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package treebank to
[nltk_data]     /home/livtollanes/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/livtollanes/nltk_data...
[nltk_data]   Unzipping taggers/maxent_treebank_pos_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/livtollanes/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/livtollanes/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [39]:
# import multiprocessing as mp

# def extract_countries_locationtagger(text):
#     if isinstance(text, str):
#         locations = locationtagger.find_locations(text=text)
#         return locations.countries
#     else:
#         return []

# # Create a Pool object with the number of cores in your machine
# pool = mp.Pool(mp.cpu_count())

# # Use the map function of the Pool object to apply the function in parallel
# follower_bios_cleaned3['countries'] = pool.map(extract_countries_locationtagger, follower_bios_cleaned3['location'])

# # Close the pool to free up system resources
# pool.close()

KeyboardInterrupt: 

In [112]:
import multiprocessing

# Take the first 100 rows of the DataFrame
subset = follower_bios_cleaned3.iloc[200:300].copy()
subset['location'] = subset['location'].str.lower()

def extract_countries_locationtagger(text):
    if isinstance(text, str):
        if 'paris' in text or 'nice' in text or 'lyon' in text:
            return ['france']
        else:
            locations = locationtagger.find_locations(text=text)
            return locations.countries
    else:
        return []

# Open the Pool
pool = multiprocessing.Pool()

results = pool.map(extract_countries_locationtagger, subset['location'])
subset.loc[:, 'countries'] = pd.Series(results, index=subset.index)

# Close the Pool
pool.close()

In [113]:
pd.set_option('display.max_rows', None)
print(subset[['location', 'countries']])

                             location      countries
200                            amiens             []
201                               NaN             []
202                            france       [france]
203       casablanca-marseille-settat             []
204                    southfield, mi             []
205        dubaï, emirats arabes unis             []
206                               NaN             []
207            republica de venezuela             []
208                               NaN             []
209                            france       [france]
210                               NaN             []
211                 россия, астрахань             []
212                          grenoble             []
213                               NaN             []
214                       bab ezzouar             []
215          buenos aires - argentina             []
216                               NaN             []
217                 région parisienne       [f

The NER so far is not working very well.
Try
- bigger language models
- more specific NER tagging tools for french locations
- manually specify a list of the 100 biggest cities in France. set country to France if they match

In [77]:
import spacy

nlp = spacy.load('en_core_web_sm')

subset2 = follower_bios_cleaned3.head(100).copy()

def extract_country_spacy(text):
    if isinstance(text, str):
        doc = nlp(text)
        countries = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
        return countries
    else:
        return []

subset2['countries'] = subset['location'].apply(extract_country_spacy)

In [90]:
print(subset2[['location', 'countries']][0:20])

                      location        countries
0                          NaN               []
1                    Venezuela      [Venezuela]
2   Valencia _ Los guayos city               []
3                          NaN               []
4                          NaN               []
5                          NaN               []
6                      Espagne               []
7                          NaN               []
8                       France         [France]
9                       France         [France]
10                      France         [France]
11                         NaN               []
12                         NaN               []
13         Bruxelles, Belgique               []
14                    Shanghai       [Shanghai]
15               Paris, France  [Paris, France]
16               Blois, France  [Blois, France]
17                       China          [China]
18                      Berlin         [Berlin]
19                         NaN          

Alternatively, idetify country using NER from spaCy. 

In [ ]:
# pip install spacy
# python -m spacy download xx_ent_wiki_sm

# import spacy

# nlp = spacy.load('xx_ent_wiki_sm')

# def extract_country_spacy(text):
#     if isinstance(text, str):
#         doc = nlp(text)
#         # Extract entities that are countries
#         countries = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
#         return countries
#     else:
#         return []
    

# df['country'] = df['location'].apply(extract_country_spacy)

In [26]:
# Inspect 'location' and 'countries' columns for rows from index 10 to 20
follower_bios_cleaned3.loc[80:100, ['location', 'countries']]

,location,countries
80,"Tourcoing, France",[France]
81,Côte d'Ivoire,[]
82,"US: 28.534544,77.151007",[]
83,Paris,[]
84,vienne,[]
85,NaN,[]
86,tunisia&qatar,[]
87,Philippines,[Philippines]
88,"Trujillo, Peru",[Peru]
89,"Nashville, TN",[]


Doing the location search with lists of countries instead

In [14]:
import geonamescache

gc = geonamescache.GeonamesCache()
all_cities = gc.get_cities()

# Filter cities by country code
cities = {k: v for k, v in all_cities.items() if v['countrycode'] == 'FR'}

# Convert the cities dictionary to a list of tuples
cities_list = [(city['name'], city['population']) for city in cities.values()]

# Sort the list by population in descending order and take the first 100
biggest_cities = sorted(cities_list, key=lambda x: x[1], reverse=True)[:650]

# Print the 100 biggest cities
for city, population in biggest_cities:
    print(city, population)

Paris 2138551
Marseille 870731
Lyon 522969
Toulouse 493465
Nice 342669
Nantes 318808
Strasbourg 274845
Bordeaux 260958
Montpellier 248252
Rouen 234475
Lille 234475
Rennes 220488
Reims 196565
Le Havre 185972
Cergy-Pontoise 183430
Saint-Étienne 176280
Toulon 168701
Angers 168279
Grenoble 158552
Dijon 158002
Nîmes 148236
Clermont-Ferrand 147865
Aix-en-Provence 146821
Saint-Quentin-en-Yvelines 146598
Brest 144899
Le Mans 144515
Amiens 143086
Tours 141621
Limoges 141176
Villeurbanne 131445
Besançon 128426
Metz 123914
Orléans 116269
Mulhouse 111430
Montreuil 111240
Perpignan 110706
Caen 110624
Boulogne-Billancourt 108782
Nancy 105058
Lyon 03 102725
Argenteuil 101475
Roubaix 98828
Tourcoing 98656
Saint-Denis 96128
Avignon 89769
Marseille 13 89316
Asnières-sur-Seine 86742
Nanterre 86719
Lyon 08 86154
Poitiers 85960
Versailles 85416
Courbevoie 85158
Créteil 84833
Pau 82697
Lyon 07 82573
Colombes 82300
Vitry-sur-Seine 81001
Aulnay-sous-Bois 80615
Marseille 08 78837
Marseille 15 77770
Marseille 0

In [13]:
len(biggest_cities)

649